<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/Recent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
lab = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/Lab.csv')

In [ ]:
df0 = pd.DataFrame(lab)

In [ ]:
import pandas as pd
df0 = pd.DataFrame(lab)
df = pd.DataFrame(df0)
df = df[['PatientUID', 'ResultItemName', 'ResultValue', 'CWhen']]
df['ResultItemName'] = df['ResultItemName'].replace(['CREATININE, 32202'], 'Creatinine')
df = df[df['ResultItemName'].isin(['eGFR(CKD-EPI) age >=18', 'Creatinine'])]
pivot = df.pivot(index=['PatientUID', 'CWhen'], columns='ResultItemName', values='ResultValue').reset_index()
sort = pivot.sort_values('CWhen').groupby('PatientUID').tail(1)
sort[['Creatinine', 'eGFR(CKD-EPI) age >=18']] = sort.groupby('PatientUID')[['Creatinine', 'eGFR(CKD-EPI) age >=18']].apply(lambda x: x.ffill().bfill())
result = sort.rename(columns={'Creatinine': 'Recent_Creatinine', 'eGFR(CKD-EPI) age >=18': 'Recent_eGFR', 'CWhen':'Recent_Date'})

<ipython-input-54-97383c362ca0>:9: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  sort[['Creatinine', 'eGFR(CKD-EPI) age >=18']] = sort.groupby('PatientUID')[['Creatinine', 'eGFR(CKD-EPI) age >=18']].apply(lambda x: x.ffill().bfill())


In [ ]:
# Fill the NaN values
import numpy as np

# Fill specific NaN values
result['Recent_eGFR'] = result['Recent_eGFR'].mask(result['PatientUID'] == 21282, result['Recent_eGFR'].fillna(55.50))
result['Recent_eGFR'] = result['Recent_eGFR'].mask(result['PatientUID'] == 173101, result['Recent_eGFR'].fillna(72.85))

result['Recent_Creatinine'] = result['Recent_Creatinine'].mask(result['PatientUID'] == 792446, result['Recent_Creatinine'].fillna(1.74))

# Verify if there are still any null values
print(result.isnull().any())

ResultItemName
PatientUID           False
Recent_Date          False
Recent_Creatinine    False
Recent_eGFR          False
dtype: bool


In [4]:
import pandas as pd
merge_df = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/SummaryFile/WideTable/WideTable.xlsx')

In [5]:
merge_df.shape

(52619, 84)

In [6]:
import pandas as pd
visit = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/Visit.csv')

<ipython-input-6-878c0ca80c69>:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  visit = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/Visit.csv')


In [8]:
# Step 1: Sort the dataframe by 'PatientVisitUID' to ensure the latest location comes last for each patient
visit.sort_values(['PatientUID', 'PatientvisitUID'], ascending=[True, False], inplace=True)
visit.head()

,PatientUID,PatientvisitUID,Sex,Age,ENTY,NATNL,StartDTTM,EndDttm,Location,PricipalProblemCode,PricipalProblemName
1275415,2299,10020327,หญิง,79,Out Patient,ไทย,2565-11-03 07:53:19.973,2565-11-04 05:01:01.637,เวชปฏิบัติทั่วไป (OPD)-D03,370992007,Dyslipidemia
1259554,2299,9897598,หญิง,79,Out Patient,ไทย,2565-09-26 10:06:28.800,2565-09-27 05:01:02.013,จักษุ (OPD)-D01,39450006,Senile cataract
1257582,2299,9881388,หญิง,79,Out Patient,ไทย,2565-09-21 08:21:28.657,2565-09-22 05:01:01.827,จิตเวชสูงอายุ (คลินิก)-D1002,F00,Dementia in Alzheimer s disease (G30.-+)
1230572,2299,9671692,หญิง,79,Out Patient,ไทย,2565-07-21 07:39:26.387,2565-07-22 05:01:01.480,เวชปฏิบัติทั่วไป (OPD)-D03,59621000,Essential hypertension
1221955,2299,9604695,หญิง,79,Out Patient,ไทย,2565-06-29 07:38:18.900,2565-06-30 05:01:01.980,จิตเวชสูงอายุ (คลินิก)-D1002,F411,Generalized anxiety disorder


In [9]:
# Step 2: Get the latest location for each patient
latest_location = visit.groupby('PatientUID').first().reset_index()[['PatientUID', 'Location']]
latest_location

,PatientUID,Location
0,2299,เวชปฏิบัติทั่วไป (OPD)-D03
1,2304,ความดันโลหิตสูงวิกฤต (คลินิก)-DV193
2,2312,บูรณาการนอกเวลา (คลินิก)-OD191
3,2354,หน่วยคลินิกพิเศษศูนย์บริการทางการแพทย์ชั้นเลิศ
4,2362,บูรณาการนอกเวลา (คลินิก)-OD191
...,...,...
22570,3421578,อายุรกรรม 9 (OPD)-D08
22571,3426284,บูรณาการนอกเวลา (คลินิก)-OD191
22572,3431501,ศัลยกรรมทั่วไป 1 (คลินิก)-D601
22573,3432056,หอผู้ป่วย 5ก-5AW


In [10]:
# Step 3: Create a dictionary mapping patient UID to the latest location
latest_location_dict = dict(zip(latest_location['PatientUID'], latest_location['Location']))
merge_df['LatestLocation'] = visit['PatientUID'].map(latest_location_dict)


In [13]:
merge_df[['LatestLocation','Location']]

,LatestLocation,Location
0,บูรณาการนอกเวลา (คลินิก)-OD191,เวชปฏิบัติทั่วไป (OPD)-D03
1,PCU สามเหลี่ยม-DV263,เวชปฏิบัติทั่วไป (OPD)-D03
2,หอผู้ป่วย AE4-E4W,เวชปฏิบัติทั่วไป (OPD)-D03
3,ห้องจ่ายยากลาง-S1PHA,เวชปฏิบัติทั่วไป (OPD)-D03
4,หน่วยผ่าตัด 4(ห้องส่องกล้อง) (Department),หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร
...,...,...
52614,ห้องจ่ายยากลาง-S1PHA,หน่วยคลินิกพิเศษศูนย์บริการทางการแพทย์ชั้นเลิศ
52615,เวชศาสตร์ฟื้นฟู (OPD)-D12,หน่วยคลินิกพิเศษศูนย์บริการทางการแพทย์ชั้นเลิศ
52616,เวชศาสตร์ฟื้นฟู (OPD)-D12,TB (คลินิก)-D707
52617,เวชศาสตร์ฟื้นฟู (OPD)-D12,โรคเลือดและต่อมน้ำเหลือง (คลินิก)-D716


In [ ]:
visit.columns

Index(['PatientUID', 'PatientvisitUID', 'Sex', 'Age', 'ENTY', 'NATNL',
       'StartDTTM', 'EndDttm', 'Location', 'PricipalProblemCode',
       'PricipalProblemName'],
      dtype='object')

In [ ]:
# Step 1: Sort the dataframe by date to ensure the latest location comes last for each patient
visit['VisitDate'] = pd.to_datetime(visit['VisitDate'])
visit.sort_values(['PatientUID', 'VisitDate'], ascending=[True, False], inplace=True)

# Step 2: Get the latest location for each patient
latest_location = visit.groupby('PatientUID').first().reset_index()[['PatientUID', 'Location']]

# Step 3: Create a dictionary mapping patient UID to the latest location
latest_location_dict = dict(zip(latest_location['PatientUID'], latest_location['Location']))

# Step 4: Use the dictionary to fill in missing values in the 'LabLocation' column
merged_df['LatestLocation'] = merged_df.apply(lambda row: latest_location_dict[row['PatientUID']] if pd.isnull(row['Location']) else row['Location'], axis=1)
